<a href="https://colab.research.google.com/github/RamSankarTheDeveloper/Question-generation-algorithm-based-on-blooms-taxonomy/blob/preprocessing/preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#note:restart the kernel if you had already imported spacy new version
#date:21/02/2023
############################
#https://youtu.be/Sp6jZFHI02Y
!git clone https://github.com/huggingface/neuralcoref.git 
import os
os.chdir("/content/neuralcoref")
!pip install -r requirements.txt
!pip install -e .  #'.' is reqd
#######################
!pip install spacy==2.3.0 
import spacy #after restarting the kernel
print(spacy. __version__)
######################
import spacy.cli 
spacy.cli.download("en")
#so that "en" works
#OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
#https://stackoverflow.com/questions/62822737/oserror-e050-cant-find-model-de-it-doesnt-seem-to-be-a-shortcut-link-a
######################
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')

#Add neural coref to SpaCy's pipe

import neuralcoref
neuralcoref.add_to_pipe(nlp)

# You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
#doc = nlp(text)

#edited_text = doc._.coref_resolved
#nouns = doc._.coref_clusters
#return edited_text, noun

In [ ]:
import pandas as pd
import numpy as np
#import spacy
nlp1 = spacy.load("en_core_web_sm")
import nltk
nltk.download('all')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import PorterStemmer
pst = PorterStemmer()


In [5]:
class preprocess:
  def __init__(self,text):
    self.text=text
    print("convert_para_to_listofsents(text)=>sentslist, convert_to_noun_phrases(sent)=>phraselist,stopwords(sent)=>list/stpwrd removed wrdlist,basewords,cnvrt_to_lwr,replace_all_corefs,convert_to_ngrams")

  def convert_para_to_listofsents(self,text):
    doc=nlp1(text)
    token_sentences=[]
    string_sentences=[]
    for sent in doc.sents:
      token_sentences.append(sent)
      string_sentences.append(sent.text)
    return  string_sentences #,token_sentences,

  def convert_to_noun_phrases(self,sentence):
    doc =nlp1(sentence)
    a=[]
    for np in doc.noun_chunks:
      a.append(np.text)
    return a

  def convert_to_tokens(self,sentence):
    #nltk
    token = word_tokenize(sentence)
    for i in token:
      token[i]=token[i].text
    return token

  def convert_to_stopwords(self,sentence):
    sp=spacy.load("en_core_web_sm")
    all_stopwords = sp.Defaults.stop_words
    text_tokens = word_tokenize(sentence)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]
    for i in tokens_without_sw:
      tokens_without_sw[i]=tokens_without_sw[i].text
    return tokens_without_sw

  def convert_to_basewords(self,sentence):
    a=[]
    doc=nlp1(sentence)
    for token in doc:
      a.append(token.lemma_)
    for i in range(len(a)):
      a[i]=pst.stem(a[i])
    return a
    
  def convert_listitems_to_lowercase(self,list1):
    for i in range(len(list1)):
      list1[i]=list1[i].cnvrt_to_lwr()
    return list1

  def replace_all_corefs(self,text): 
    #change it into different versions
    #https://stackoverflow.com/questions/6570635/installing-multiple-versions-of-a-package-with-pip
    doc = nlp(text)
    edited_text = doc._.coref_resolved
    nouns = doc._.coref_clusters
    for i in range(len(nouns)):
      nouns[i]=str(nouns[i]).split(":")[0]
    return edited_text #, nouns

  def convert_to_ngrams(self,sentence, n):
    manygrams = list(ngrams(sentence.split(), n))
    for i in manygrams:
      manygrams[i]=manygrams[i].text
    return manygrams
  


In [ ]:
##############################################_diff_##############################################

In [20]:

class cls_diff:
 

  def __init__(self, paragraph):
    self.paragraph = paragraph
    cls_diff.prepare_seperating_and_synonyms_words()
    cls_diff.prepare_paragraph_to_0basewords_of_sentences_listoflist0()
    cls.


    

def prepare_seperating_and_synonyms_words(self):
  #prepare difference_synonyms_basewords_list and seperator_words for matching (check_0difference_synonyms_basewords_list0_present_in_0basewords_of_sentences_listoflist0():)
  obj_diff = preprocess(cls_diff.difference_synonyms_string)
  cls_diff.difference_synonyms_basewords_list = obj_diff.convert_to_basewords(cls_diff.difference_synonyms_string)
  cls_diff.seperator_words = obj_diff.convert_to_basewords("and versus vs ,")

def prepare_paragraph_to_0basewords_of_sentences_listoflist0(self):
  #text preperation into basewords for matching (check_0difference_synonyms_basewords_list0_present_in_0basewords_of_sentences_listoflist0():)
  text = self.paragraph
  preprocess_obj = preprocess(text)
  coref_text = preprocess_obj.replace_all_corefs(text)
  sentences_list=preprocess_obj.convert_para_to_listofsents(text)
  basewords_of_sentences_listoflist=[]
  for each_sentence in sentences_list:
    basewords_of_sentences_listoflist.append(preprocess_obj.convert_to_basewords(each_sentence))
  cls_diff.basewords_of_sentences_listoflist = basewords_of_sentences_listoflist

def extract_sentences_containing_synonyms(self):
  #returns the index if the synonym of "difference" is present in any of sentences. 
  # variables: #synonyms of the word "difference" = difference_synonyms_basewords_list , sentences = basewords_of_sentences_listoflist
  basewords_of_sentences_listoflist = cls_diff.basewords_of_sentences_listoflist
  for index in range(len(basewords_of_sentences_listoflist)):
    for each_synonym in difference_synonyms_basewords_list:
      if each_synonym in basewords_of_sentences_listoflist[index]:
        return index
          
def split_sentences_with_between_etc():
  if "between" in sentences_list[i].lower():
    betw_split_sentence=str(sentences_list[i].split("between")[1])
    words_sentence_listofstrngs=betw_split_sentence.split()
      #regex

def split_sententences_with_and_etc():
  import re
#test_list = ['and', 'versus']
test_list = cls_diff.seperator_words
betw_split_sentence = 'the main difference between power of z angle, x angle and y  angle is very interesting to teach'
# code to split the line with 'diff' with keywords and and comma
if re.search(r'\b({})\b'.format("|".join(test_list)),betw_split_sentence):
  splitted_parts_lstofstrngs = re.sub("[\s]and[\s]", "~",betw_split_sentence )
  splitted_parts_lstofstrngs = re.sub(",[\s]", "~", splitted_parts_lstofstrngs)
  splitted_parts_lstofstrngs = re.sub("[\s]versus[\s]", "~", splitted_parts_lstofstrngs)
  splitted_parts_lstofstrngs = re.sub("[\s]vs[\s]", "~", splitted_parts_lstofstrngs)
  splitted_parts_lstofstrngs = str(splitted_parts_lstofstrngs.split("between")[1])
  splitted_parts_lstofstrngs = re.split("~",splitted_parts_lstofstrngs)
print(splitted_parts_lstofstrngs)

#code to extract dependencies from parts splitted by seperating words.
words_and_deps_ofsplit_asdict={}
dep_pattern_ofsplits_asstrng=""
words_and_deps_ofsplits_aslistofdict=[]
for each_splitted_part in splitted_parts_lstofstrngs:
  for token in word_tokenize(each_splitted_part):
    words_and_deps_ofsplit_asdict[nltk.pos_tag([token])[0][0]]=nltk.pos_tag([token])[0][1]  #0=>word, 1=>dep
    dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng+nltk.pos_tag([token])[0][1]
  words_and_deps_ofsplits_aslistofdict.append(words_and_deps_ofsplit_asdict)
  dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng+"~"
  words_and_deps_ofsplit_asdict={}
print(words_and_deps_ofsplits_aslistofdict)
dep_pattern_ofallsplits_asstrng=dep_pattern_ofallsplits_asstrng[:-1]
print(dep_pattern_ofallsplits_asstrng)
dep_pattern_ofallsplits_aslstofstrngs=dep_pattern_ofallsplits_asstrng.split("~")
print(dep_pattern_ofallsplits_aslstofstrngs)

#choose smallest part of seperated line parts
smallest_diff_line=dep_pattern_ofallsplits_aslstofstrngs[0]
for i in dep_pattern_ofallsplits_aslstofstrngs[1:]:
  if len(i)<len(smallest_diff_line):
    smallest_diff_line=i
print(smallest_diff_line)

def create_all_ordered_compinations_of_word (self,word):
  #code to get each ordered compination of a sentence to check for a pattern from in descending order of length
  for incr in range(len(word)):
    print("===")
    decr=len(word)-incr-1
    decr_cp=decr
    #print(line[0:k])
    sub_incr=0
    while decr_cp !=len(word):
      function check_word_is_present_in_all()
      #print(word[sub_incr:decr_cp+1]) #activate this line to print all ordered combinations of word in the terminal
      decr_cp=decr_cp+1
      sub_incr=sub_incr+1

function to compare,output=> a pettern
function to select that pattern


#diff_list=["contrast","difference","distinction","contrary","different","differing","unlike","differ","unlike","versus","vs","v/s"]
#diff_maybe_list=["the opposite","the contrary","the reverse","compare","on the other hand","while","whereas","but"]
seperator_words=["and",",","versus","vs"]
#similarity_list=["resemblance similarity parallels analogy similar, analogous, common, comparable, identical, same, resemble, correspond, similarly, likewise, same, like, as"]



convert_para_to_sents(text)=>sentslist, convert_to_noun_phrases(sent)=>phraselist,stopwords(sent)=>list/stpwrd removed wrdlist,basewords,cnvrt_to_lwr,replace_all_corefs,convert_to_ngrams


In [17]:
text="many have splitted into two after world war two. There is still a sharp contrast between East and West Germany.One is ruled by western countries and one is ruled by soviet union"





#check if word is in sent
for i in range(len(basewords_of_sentences_listoflist)):
  for diffword in difference_synonyms_basewords_list:
    if diffword in basewords_of_sentences_listoflist[i]:
      #function1(basewords_of_sentences_listoflist[i],basewords_of_sentences_listoflist[i-1],basewords_of_sentences_listoflist[i+1])
      if "between" in sentences_list[i].lower():
        betw_split_sentence=str(sentences_list[i].split("between")[1])
        words_sentence=betw_split_sentence.split()
#regex


convert_para_to_sents(text)=>sentslist, convert_to_noun_phrases(sent)=>phraselist,stopwords(sent)=>list/stpwrd removed wrdlist,basewords,cnvrt_to_lwr,replace_all_corefs,convert_to_ngrams
convert_para_to_sents(text)=>sentslist, convert_to_noun_phrases(sent)=>phraselist,stopwords(sent)=>list/stpwrd removed wrdlist,basewords,cnvrt_to_lwr,replace_all_corefs,convert_to_ngrams


In [12]:
import re

test_list = ['and', 'versus']
text1 = 'the main difference between power of z angle, x angle and y  angle is very interesting to teach'

# code to split the line with 'diff' with keywords and and comma
if re.search(r'\b({})\b'.format("|".join(test_list)),text1):
  txt=re.sub("[\s]and[\s]", "~",text1 )
  txt=re.sub(",[\s]", "~", txt)
  txt=re.sub("[\s]versus[\s]", "~", txt)
  txt=re.sub("[\s]vs[\s]", "~", txt)
  txt=str(txt.split("between")[1])
  txt=re.split("~",txt)
print(txt)



[' power of z angle', 'x angle', 'y  angle is very interesting to teach']


In [13]:

#code to extract dependencies
dict_a={}
pattern_line=""
list_dict_a=[]
for txtitem in txt:
  for token in word_tokenize(txtitem):
    dict_a[nltk.pos_tag([token])[0][0]]=nltk.pos_tag([token])[0][1]
    pattern_line=pattern_line+nltk.pos_tag([token])[0][1]
  list_dict_a.append(dict_a)
  pattern_line=pattern_line+"~"
  dict_a={}
print(list_dict_a)
pattern_line=pattern_line[:-1]   #to remove the last tilda
print(pattern_line)  
sep_diff_lines=pattern_line.split("~")  #note:probably could seperate these in the previous for loop itself and simplify the code
print(sep_diff_lines)

#choose smallest part of seperated line parts
smallest_diff_line=sep_diff_lines[0]
for i in sep_diff_lines[1:]:
  if len(i)<len(smallest_diff_line):
    smallest_diff_line=i
print(smallest_diff_line)

      

[{'power': 'NN', 'of': 'IN', 'z': 'NN', 'angle': 'NN'}, {'x': 'NN', 'angle': 'NN'}, {'y': 'NN', 'angle': 'NN', 'is': 'VBZ', 'very': 'RB', 'interesting': 'VBG', 'to': 'TO', 'teach': 'NN'}]
NNINNNNN~NNNN~NNNNVBZRBVBGTONN
['NNINNNNN', 'NNNN', 'NNNNVBZRBVBGTONN']
NNNN


In [1]:
#class_attributes
  difference_synonyms_string = "contrast difference distinction contrary different differing unlike differ unlike versus vs"
  obj_diff=preprocess(difference_synonyms_string)
  difference_synonyms_basewords_list=obj_diff.convert_to_basewords(difference_synonyms_string)

In [ ]:
###################################################################_similarities_#######################################################

In [21]:
text

'many have splitted into two after world war two. There is still a sharp contrast between East and West Germany.One is ruled by western countries and one is ruled by soviet union'

In [24]:
objsmple=preprocess(text)
#coref_text=objsmple.replace_all_corefs(text)
sentences_list=obj.convert_para_to_listofsents(text)
many_sent_basewords=[]
for sent_each in sentences_list:
  many_sent_basewords.append(obj.convert_to_basewords(sent_each))
many_sent_basewords

convert_para_to_sents(text)=>sentslist, convert_to_noun_phrases(sent)=>phraselist,stopwords(sent)=>list/stpwrd removed wrdlist,basewords,cnvrt_to_lwr,replace_all_corefs,convert_to_ngrams


AttributeError: ignored

In [1]:
dict_a={}
pattern_line=""
list_dict_a=[]
for txtitem in txt:
  for token in word_tokenize(txtitem):
    dict_a[nltk.pos_tag([token])[0][0]]=nltk.pos_tag([token])[0][1]
    pattern_line=pattern_line+nltk.pos_tag([token])[0][1]
  list_dict_a.append(dict_a)
  pattern_line=pattern_line+"~"
  dict_a={}
print(list_dict_a)
pattern_line=pattern_line[:-1]
print(pattern_line)
sep_diff_lines=pattern_line.split("~")
print(sep_diff_lines)

NameError: ignored